In [ ]:
import sys

import numpy as np
from river.decomposition import OnlineDMD, OnlineSVD
from river.preprocessing import Hankelizer
from river.utils import Rolling

Rolling.learn_one = Rolling.update
sys.path.append("../")

from functions.datasets import load_nprs43, load_nprs44  # noqa: E402
from functions.chdsubid import SubIDChangeDetector  # noqa: E402
from functions.plot import plot_chd  # noqa: E402

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# [donotremove]
# Read data
X = load_nprs44()

# Initialize Hankelizer
hn = 80
hankelizer = Hankelizer(hn)

# Initialize Transformer
r = 2
init_size = 300
odmd = OnlineDMD(
    r=r,
    initialize=init_size,
    w=1.0,
    eig_rtol=1e-1,
)
odmd = Rolling(
    OnlineDMD(
        r=r,
        initialize=init_size,
        w=1.0,
        exponential_weighting=False,
        eig_rtol=1e-1,
    ),
    init_size + 1,
)
# Ref
osvd = OnlineSVD(r, initialize=init_size, force_orth=False)
# Initialize Change-Point Detector
ref_size = 100
test_size = 100
subid_dmd = SubIDChangeDetector(
    odmd,
    ref_size=ref_size,
    test_size=test_size,
    grace_period=init_size + test_size + 1,
)
# Ref
subid_svd = SubIDChangeDetector(
    osvd,
    ref_size=ref_size,
    test_size=test_size,
    grace_period=init_size + test_size + 1,
)

# Build pipeline
pipeline_dmd = hankelizer | subid_dmd
pipeline_svd = Hankelizer(hn) | subid_svd

# Prepare arrays for storing results
scores_dmd = np.zeros(X.shape[0], dtype=float)
scores_svd = np.zeros(X.shape[0], dtype=float)
scores_dmd_diff = np.zeros(X.shape[0], dtype=complex)
dist = np.zeros((X.shape[0], 2), dtype=complex)
modes = np.empty((X.shape[0], hn, r), dtype=complex)
mean_modes = np.empty(X.shape[0], dtype=complex)
svs = np.zeros((X.shape[0], r, r), dtype=float)
us = np.zeros((X.shape[0], hn, r), dtype=float)
As = np.zeros((X.shape[0], r, r), dtype=float)

# Run pipeline
for i, (x) in enumerate(X):
    # print every 1000
    if (i + 1) % 1000 == 0:
        print(f"\rProcessed {i + 1}/{len(X)}", end="")

    pipeline_dmd.learn_one({"x": x})
    pipeline_svd.learn_one({"x": x})

    scores_dmd[i] = subid_dmd.score
    scores_svd[i] = subid_svd.score
    if hasattr(subid_dmd.subid, "modes"):
        modes[i, :, :] = odmd.modes
        mean_modes[i] = modes[i, :, :].mean()
        svs[i, :, :] = odmd._svd._S
        us[i, :, :] = odmd._svd._U
        As[i, :, :] = odmd.A
        dist[i, :] = subid_dmd.distances

    scores_dmd_diff[i] = dist[i, 1] - dist[i, 0]

# Plot results
fig, axs = plot_chd(
    [X, scores_dmd, scores_dmd_diff.real, scores_svd],
    labels=["X", "DMD", "DMD_diff", "SVD"],
    idx_start=15500,
    idx_end=22000,
    idx_in_start=17000,  # Original 16190
    idx_in_end=19000,  # Original 16344
)

fig_name = (
    f"results/.nprs/nprs44-chd_r{r}-"
    f"{f'roll_{odmd.window_size}' if isinstance(odmd, Rolling) else 'noroll'}-"
    f"dmd_w{odmd.w}-h{hn}-imag.png"
)
fig.savefig(fig_name)

In [ ]:
# [donotremove]
# Read data
X = load_nprs43()

# Initialize Hankelizer
hn = 80
hankelizer = Hankelizer(hn)

# Initialize Transformer
r = 2
init_size = 300
odmd = OnlineDMD(r=r, initialize=init_size, w=1.0)
odmd = Rolling(
    OnlineDMD(
        r=r,
        initialize=init_size,
        w=1.0,
        exponential_weighting=False,
        eig_rtol=1e-1,
    ),
    init_size + 1,
)
# Ref
osvd = OnlineSVD(r, initialize=init_size, force_orth=False)
# Initialize Change-Point Detector
ref_size = 100
test_size = 100
subid_dmd = SubIDChangeDetector(
    odmd,
    ref_size=ref_size,
    test_size=test_size,
    grace_period=init_size + test_size + 1,
)
# Ref
subid_svd = SubIDChangeDetector(
    osvd,
    ref_size=ref_size,
    test_size=test_size,
    grace_period=init_size + test_size + 1,
)

# Build pipeline
pipeline_dmd = hankelizer | subid_dmd
pipeline_svd = Hankelizer(hn) | subid_svd

# Prepare arrays for storing results
scores_dmd = np.zeros(X.shape[0], dtype=float)
scores_svd = np.zeros(X.shape[0], dtype=float)
scores_dmd_diff = np.zeros(X.shape[0], dtype=complex)
dist = np.zeros((X.shape[0], 2), dtype=complex)
modes = np.empty((X.shape[0], hn, 2), dtype=complex)
mean_modes = np.empty(X.shape[0], dtype=complex)
svs = np.zeros((X.shape[0], 2, 2), dtype=float)
us = np.zeros((X.shape[0], hn, 2), dtype=float)
As = np.zeros((X.shape[0], 2, 2), dtype=float)

# Run pipeline
for i, (x) in enumerate(X):
    # print every 1000
    if (i + 1) % 1000 == 0:
        print(f"\rProcessed {i + 1}/{len(X)}", end="")

    pipeline_dmd.learn_one({"x": x})
    pipeline_svd.learn_one({"x": x})

    scores_dmd[i] = subid_dmd.score
    scores_svd[i] = subid_svd.score
    if hasattr(subid_dmd.subid, "modes"):
        modes[i, :, :] = odmd.modes
        mean_modes[i] = modes[i, :, :].mean()
        svs[i, :, :] = odmd._svd._S
        us[i, :, :] = odmd._svd._U
        As[i, :, :] = odmd.A
        dist[i, :] = subid_dmd.distances

    scores_dmd_diff[i] = dist[i, 1] - dist[i, 0]

# Plot results
fig, axs = plot_chd(
    [X, scores_dmd, scores_dmd_diff, scores_svd],
    labels=["X", "DMD", "DMD_diff", "SVD"],
    idx_start=10000,
    idx_end=14000,
)

fig_name = (
    f"results/.nprs/nprs43-chd_r{r}-"
    f"{f'roll_{odmd.window_size}' if isinstance(odmd, Rolling) else 'noroll'}-"
    f"dmd_w{odmd.w}-h{hn}-imag-corr.png"
)
fig.savefig(fig_name)